## Cálculo del grupo de Galois de un polimonio irreducible de grado 5

#### Importación de librerías necesarias

In [62]:
from sympy import *
from sympy.abc import t
import numpy as np
from IPython.display import display, Latex

import pandas as pd

### Funciones auxiliares

Se definen dos funciones auxiliares para el cálculo de los dos invariantes más impotantes que se usan. Así, se define primero la función _Discriminante_ para calcular el valor del discriminante de un polinomio de grado 5 a partir de sus coeficientes. Despueés, se define la función _Resolvente_Séxtica_ que devuelve el polinomio Rfp, que es la resolvente séxtica especializada en f, a partir de los coeficientes de f y de su discrimante. Además, de la resolvente, la función devuelve el término independiente de ésta pues también se necesita.

In [63]:
def _Discriminante(a):
    a0 = a[0]; a1 = a[1]; a2 = a[2]; a3 = a[3]; a4 = a[4]
    m = [[a0,  a1,  a2,  a3,  a4,   1,   0,   0,   0],
        [  0,  a0,  a1,  a2,  a3,  a4,   1,   0,   0],
        [  0,   0,  a0,  a1,  a2,  a3,  a4,   1,   0],
        [  0,   0,   0,  a0,  a1,  a2,  a3,  a4,   1],
        [ a1,2*a2,3*a3,4*a4,   5,   0,   0,   0,   0],
        [  0,  a1,2*a2,3*a3,4*a4,   5,   0,   0,   0],
        [  0,   0,  a1,2*a2,3*a3,4*a4,   5,   0,   0],
        [  0,   0,   0,  a1,2*a2,3*a3,4*a4,   5,   0],
        [  0,   0,   0,   0,  a1,2*a2,3*a3,4*a4,   5]]
    m = Matrix(m)
    Delta = m.det()
    return Delta

def _Resolvente_Sextica(a, Delta):
    s1 = -a[4]; s2 = a[3]; s3 = -a[2]; s4 = a[1]; s5 = -a[0]

    A2 = 8*s1 * s3 - 3*s2**2 - 20*s4

    A4 = 3*s2**4 - 16*s1*s2**2*s3 + 16*s1**2*s3**2 + 16*s2*s3**2 + 16*s1**2*s2*s4 
    A4 += - 8*s2**2*s4 - 112*s1*s3*s4 + 240*s4**2 - 64*s1**3*s5 + 240*s1*s2*s5 - 400*s3*s5

    A6 = 8*s1*s2**4*s3 - s2**6 - 16*(s1*s2*s3)**2 - 16*s2**3*s3**2 + 64*s1*s2*s3**3 - 64*s3**4 
    A6 += -16*s1**2*s2**3*s4 + 28*s2**4*s4 + 64*s1**3*s2*s3*s4 - 112*s1*s2**2*s3*s4 - 128*s1**2*s3**2*s4 
    A6 += 224*s2*s3**2*s4 - 64*s1**4*s4**2 + 224*s1**2*s2*s4**2 - 176*s2**2*s4**2 - 64*s1*s3*s4**2 
    A6 += 320*s4**3 + 48*s1*s2**3*s5 - 192*s1**2*s2*s3*s5 - 80*s2**2*s3*s5 + 640*s1*s3**2*s5 
    A6 += 384*s1**3*s4*s5 - 640*s1*s2*s4*s5 - 1600*s3*s4*s5 - 1600*s1**2*s5**2 + 4000*s2*s5**2
    
    RS1 = 1
    RS2 = 2*A2
    RS3 = A2**2 + 2*A4
    RS4 = 2*(A2*A4 + A6) 
    RS5 = 2*A2*A6 + A4**2
    RS6 = 2*A4*A6 - 2**10* Delta
    RS7 = A6**2;
    Rfp = poly(RS1*t**6 + RS2*t**5 + RS3*t**4 + RS4*t**3 + RS5*t**2 + RS6*t + RS7, t, domain='Q')
    return Rfp, RS7


### Función principal

Ahora podemos definir la función principal que determina el grupo de Galois de un polinomio irreducible de grado 5 dado. La función comprueba primero que el polinomio sea en efecto irreducible y en tal caso calcula los invariantes necesarios para decidir su grupo de Galois. En particular, comprueba si la raíz cuadrada del discriminante es racional (equivale a que sea entera), si la resolvente séxtica tiene alguna raíz racional y si f tiene todas las raíces reales.

In [64]:
def Calcular_grupo_Galois (a, verv=True):
    f = poly(a[5]*t**5 + a[4]*t**4 + a[3]*t**3 + a[2]*t**2 + a[1]*t + a[0], t, domain='Q')
    if a[0] == 0 or 0 in [f(d) for d in divisors(a[0])]:
        if verv:
            print('Poliniomio reducible')
        return '-' # Reducible
    
    Delta = _Discriminante(a)
    delta = sqrt(Delta) if Delta >= 0 else 0.1
    Rfp, TermIndep = _Resolvente_Sextica(a, Delta)
    DivTI = divisors(TermIndep)
    
    Criterio1 = np.floor(delta) == np.ceil(delta)
    Criterio2 = TermIndep == 0 or 0 in [Rfp(d) for d in DivTI]
    Criterio3 = False not in [r.is_real for r in nroots(f)]
    
    if Criterio1 and Criterio2 and Criterio3:
        if verv:
            print('Criterio 1: Discriminante con raíz cuadrada racional')
            print('Criterio 2: Resolvente séxtica con raíz racional')
            print('Criterio 3: Todas las raíces de f son reales')
            display(Latex(f'$G_\mathbb{{Q}}(f) \in \lbrace \mathbb{{Z}}_5, \mathcal{{D}}_5 \}}$'))
        Grupo = 'Z5/D5' 
    elif Criterio1 and Criterio2:
        if verv:
            print('Criterio 1: Discriminante con raíz cuadrada racional')
            print('Criterio 2: Resolvente séxtica con raíz racional')
            print('Criterio 3: f tiene raíces no reales')
            display(Latex(f'$G_\mathbb{{Q}}(f) = \mathcal{{D}}_5$'))
        Grupo = 'D5' 
    elif Criterio1:
        if verv:
            print('Criterio 1: Discriminante con raíz cuadrada racional')
            print('Criterio 2: Resolvente séxtica sin raíz racional')
            display(Latex(f'$G_\mathbb{{Q}}(f) = \mathcal{{A}}_5$'))
        Grupo = 'A5'
    elif Criterio2:
        if verv:
            print('Criterio 1: Discriminante con raíz cuadrada irracional')
            print('Criterio 2: Resolvente séxtica con raíz racional')
            display(Latex(f'$G_\mathbb{{Q}}(f) = \mathcal{{F}}_5$'))
        Grupo = 'F5'
    else:
        if verv:
            print('Criterio 1: Discriminante con raíz cuadrada irracional')
            print('Criterio 2: Resolvente séxtica sin raíz racional')
            display(Latex(f'$G_\mathbb{{Q}}(f) = \mathcal{{S}}_5$'))
        Grupo = 'S5'
    
    return Grupo      

A modo de ejemplo se utiliza el código desarrollado para obtener el grupo de Galois de 5 polinomios, cada uno de los cuales resulta tener uno de los posibles grupos de Galois.

In [65]:
fa1 = [-1, -2,  5,  2, -4,  1]
fa2 = [-1, -2, -2, -1,  0,  1]
fa3 = [ 1,  1,  4,  2,  1,  1]
fa4 = [-2, -2, -2,  0,  1,  1]
fa5 = [ 1,  1, -1, -1,  0,  1]
Calcular_grupo_Galois(fa1)
Calcular_grupo_Galois(fa2)
Calcular_grupo_Galois(fa3)
Calcular_grupo_Galois(fa4)
Calcular_grupo_Galois(fa5)

Criterio 1: Discriminante con raíz cuadrada racional
Criterio 2: Resolvente séxtica con raíz racional
Criterio 3: Todas las raíces de f son reales


<IPython.core.display.Latex object>

Criterio 1: Discriminante con raíz cuadrada racional
Criterio 2: Resolvente séxtica con raíz racional
Criterio 3: f tiene raíces no reales


<IPython.core.display.Latex object>

Criterio 1: Discriminante con raíz cuadrada irracional
Criterio 2: Resolvente séxtica con raíz racional


...

Criterio 1: Discriminante con raíz cuadrada racional
Criterio 2: Resolvente séxtica sin raíz racional


<IPython.core.display.Latex object>

Criterio 1: Discriminante con raíz cuadrada irracional
Criterio 2: Resolvente séxtica sin raíz racional


<IPython.core.display.Latex object>

'S5'

### Familias de trinomios

Por último, se calcula el grupo de Galois de las familias de trinomios t^5+at^2+b y t^5+at+b (Bring-Jerrard). Para valores de a y b enteros con valor absoluto no mayor que 10.

In [66]:
f = open('Trinomio.txt', 'w')
g = open('TrinomioBJ.txt', 'w')
f.write('a;b;G#')
g.write('a;b;G#')
for a in range(-10,11):
    for b in range(-10,11):
        G = Calcular_grupo_Galois([b,0,a,0,0,1], False)
        f.write(str(a)+';'+str(b)+';'+G+'#')
        
        G = Calcular_grupo_Galois([b,a,0,0,0,1], False)
        g.write(str(a)+';'+str(b)+';'+G+'#')
f.close()
g.close()

In [67]:
df_Trinomio   = pd.read_csv('Trinomio.txt',   sep=';', lineterminator='#')
df_TrinomioBJ = pd.read_csv('TrinomioBJ.txt', sep=';', lineterminator='#')
df   = pd.DataFrame()
dfBJ = pd.DataFrame()
for i in range(-10,11):
    df  [str(i)] = list(df_Trinomio[  df_Trinomio[  'a'] == i]['G'])    
    dfBJ[str(i)] = list(df_TrinomioBJ[df_TrinomioBJ['a'] == i]['G'])
df   =   df.set_index(pd.Index(list(range(-10,11))))
dfBJ = dfBJ.set_index(pd.Index(list(range(-10,11))))

In [68]:
def cell_format(val):
    if val=='S5':
        color = 'pink'
    elif val =='A5':
        color = 'red'
    elif val =='D5':
        color = 'yellow'
    elif val =='F5':
        color = 'cyan'
    elif val =='Z5/D5':
        color = 'grey'
    else:
        color = 'white'
    return 'background-color: %s' % color

In [69]:
df.style.applymap(cell_format)

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10
-10,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,S5,S5,S5,-,S5
-9,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,S5,S5,-,S5,S5
-8,S5,S5,S5,S5,-,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,S5,-,S5,S5,S5
-7,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,-,S5,S5,S5,S5
-6,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,-,S5,S5,S5,S5,S5
-5,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,-,S5,S5,S5,S5,S5,S5
-4,S5,S5,S5,-,S5,S5,S5,S5,S5,S5,F5,S5,S5,-,S5,S5,S5,S5,S5,S5,S5
-3,S5,S5,S5,S5,S5,D5,S5,S5,S5,S5,F5,S5,-,S5,S5,S5,S5,S5,S5,S5,S5
-2,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,-,S5,S5,S5,S5,S5,S5,S5,S5,S5
-1,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,-,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5


In [70]:
dfBJ.style.applymap(cell_format)

,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10
-10,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,S5,S5,S5,-,S5
-9,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,S5,S5,-,S5,S5
-8,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,S5,-,S5,S5,S5
-7,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,S5,-,S5,S5,S5,S5
-6,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,S5,-,S5,S5,S5,S5,S5
-5,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,S5,S5,-,S5,S5,S5,S5,S5,S5
-4,S5,S5,S5,S5,S5,A5,S5,S5,S5,S5,F5,S5,S5,-,S5,S5,S5,S5,S5,S5,S5
-3,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,S5,-,S5,S5,S5,S5,S5,S5,S5,S5
-2,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,F5,-,S5,S5,S5,S5,S5,S5,S5,S5,S5
-1,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5,-,S5,S5,S5,S5,S5,S5,S5,S5,S5,S5
